In [13]:
import boto3
from AWSConfig import *    #config details are saved in another seperate file

In [14]:
translate = boto3.client(
    service_name='translate', 
    region_name=region, 
    use_ssl=True,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,)

In [17]:
result = translate.translate_text(Text="Hello Shivam", SourceLanguageCode="en", TargetLanguageCode="hi")

In [23]:
result

{'TranslatedText': 'नमस्ते शिवम',
 'SourceLanguageCode': 'en',
 'TargetLanguageCode': 'hi',
 'ResponseMetadata': {'RequestId': '19dc3c4f-ba67-4b0d-9786-ff54f9984159',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '19dc3c4f-ba67-4b0d-9786-ff54f9984159',
   'cache-control': 'no-cache',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Fri, 08 Apr 2022 09:10:58 GMT'},
  'RetryAttempts': 0}}

In [22]:
if result:
    print(result.get('TranslatedText',""))

नमस्ते शिवम


In [25]:
import time
import boto3

def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='flac',
        LanguageCode='en-US'
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                print(
                    f"Download the transcript from\n"
                    f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}.")
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

In [28]:
transcribe_client = boto3.client('transcribe', region_name=region,aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key)
file_uri = r's3://testing-bucket-transcribe/input/Recording.m4a'
transcribe_file('Example-job', file_uri, transcribe_client)

Waiting for Example-job. Current status is IN_PROGRESS.
Job Example-job is FAILED.
